In [3]:
# Import libraries
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score, classification_report


In [7]:
path_to_td_folder = r"C:\Python310\Lib\site-packages\td"
sys.path.insert(0, path_to_td_folder)
from td.client import TDClient

In [13]:
def grab_price_data():

    # Create a new session
    TDSession = TDClient(account_number = 'adammahdi10@hotmail.com',
                         account_password = 'Yarayara100+',
                         consumer_id = 'FQX83ZV1WGSC0VQRUTNDFR5WDICVMSYM',
                         redirect_uri = 'http://127.0.0.1')

    # Login to the session
    TDSession.login()

        # Defining the list of tickers
        #JPM = JP Morgan, COST = Costco, HD = Home Depot
    list_tickers = ['JPM', 'COST', 'IBM', 'HD', 'ARWR']

    # I need to store multiple result sets.
    price_history = []

    for ticker in list_tickers:

        # Retrieve the daily price history for 1 year
        price_history = TDSession.get_price_history(symbol = ticker, 
                                                    periodType = 'year', 
                                                    period = 2, 
                                                    frequency = 1, 
                                                    frequencyType = 'daily', 
                                                    needExtendedHoursData = False)

        # retrieve candles, and add towards the list.
        for candle in price_history['candles']:
            candle['symbol'] = price_history['symbol']
            price_history.append(candle)

    # put the data to a CSV file, without having an index column
    price_data = pd.DataFrame(price_history).to_csv('price_data.csv', index_label = False)

In [14]:
if os.path.exists('price_data.csv'):

    # Load data from csv file
    price_data = pd.read_csv('price_data.csv')

else:

    # retrieve and store data
    grab_price_data()

    # Load the data
    price_data = pd.read_csv('price_data.csv')

# Display the initial data
price_data.head()

,close,datetime,high,low,open,symbol,volume
0,105.51,1513144800000,107.11,105.48,106.70,JPM,15020293
1,104.66,1513231200000,106.45,104.64,106.12,JPM,13034232
2,106.14,1513317600000,106.52,105.20,105.79,JPM,29350598
3,106.96,1513576800000,107.63,106.48,107.19,JPM,12713148
4,106.51,1513663200000,107.49,106.44,107.35,JPM,12149937


In [16]:
#reorganising data columns
price_data = price_data[['symbol','datetime','close','high','low','open','volume']]
price_data.sort_values(by = ['symbol','datetime'], inplace = True)
price_data['change_in_price'] = price_data['close'].diff()

price_data.head()

,symbol,datetime,close,high,low,open,volume,change_in_price
2016,ARWR,1513144800000,3.40,3.410,3.0100,3.30,2037395,NaN
2017,ARWR,1513231200000,3.15,3.420,3.0500,3.41,1702102,-0.25
2018,ARWR,1513317600000,3.14,3.200,3.0800,3.19,1305646,-0.01
2019,ARWR,1513576800000,3.23,3.395,3.1400,3.15,1254305,0.09
2020,ARWR,1513663200000,3.26,3.380,3.2271,3.25,1128590,0.03


In [17]:
# identifying rows where the symbol changes
mask = price_data['symbol'] != price_data['symbol'].shift(1)

# For those rows, make the value null
price_data['change_in_price'] = np.where(mask == True, np.nan, price_data['change_in_price'])

# print the rows that have a null value
price_data[price_data.isna().any(axis = 1)]

,symbol,datetime,close,high,low,open,volume,change_in_price
2016,ARWR,1513144800000,3.40,3.41,3.01,3.30,2037395,NaN
504,COST,1513144800000,188.28,190.12,187.80,188.53,3062856,NaN
1512,HD,1513144800000,183.03,183.67,182.00,182.01,5177363,NaN
1008,IBM,1513144800000,153.91,156.73,153.89,156.60,5661618,NaN
0,JPM,1513144800000,105.51,107.11,105.48,106.70,15020293,NaN
